# **Activity VAE**

## i. Imports

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import gc
import tqdm

## ii. General setup

In [2]:
# Directories
basedir = os.getcwd()
datadir = os.path.join(basedir, 'data')

# Training hyperparameters
N_ROUNDS = 15
N_CENTRES = 4
N_EPOCHS = 5
N_FEATURES = 19
BATCH_SIZE = 2
LATENT_DIM = 5

# GPU settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    gc.collect()
    torch.cuda.empty_cache()

## **1. Load data**

In [3]:
# Custom class to create pytorch Dataset of activity data
class ActivityDataset(Dataset):
    """ Custom PyTorch Dataset 
        of activity data.
    """

    def __init__(self, data_array):
        """
        Arguments:
            data_array (array): NumPy array of data.
        """
        self.data_array = data_array

    def __len__(self):
        return len(self.data_array)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = self.data_array[idx]
        label = sample[-1]
        sample = sample[:-1]
        sample = torch.Tensor(sample)

        return sample, label

In [4]:
# Load training and test
data_file = 'date-val_False.npz'
data = np.load(os.path.join(datadir, data_file))
train_data = data['X_train']
test_data = data['X_test']

# Make dataframes
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Print some statisics
print(f'Training data points: {train_data.shape[0]}')
print(f'Test data points: {test_data.shape[0]}')
assert train_data.shape[1] == test_data.shape[1], f'Training and test data have different number of features ({train_data.shape[1]}, {train_data.shape[1]})'
print(f'Number of features: {train_data.shape[1]}')

# Create Datasets
train_dataset = ActivityDataset(train_data)
test_dataset = ActivityDataset(test_data)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, num_workers=0)

Training data points: 25979
Test data points: 5081
Number of features: 20


## **2. Define VAE model**

In [14]:
class VAE(nn.Module):
    """ Convulutional VAE model."""
    def __init__(self,
                 in_features=20,
                 latent_dim=5):
        super(VAE, self).__init__()

        # Assign variables
        self.in_features = in_features
        self.latent_dim = latent_dim

        # Build Encoder
        modules = []
        modules.append(
            nn.Sequential(
                nn.Linear(self.in_features, self.in_features // 2),
                nn.LeakyReLU()
                )
        )
        self.encoder = nn.Sequential(*modules)

        # Linear layers defining distribution parameters
        self.fc_mu = nn.Linear(self.in_features // 2, self.latent_dim)
        self.fc_var = nn.Linear(self.in_features // 2, self.latent_dim)

        # Build Decoder
        modules = []
        modules.append(
            nn.Sequential(
                nn.Linear(self.latent_dim, self.in_features //2),
                nn.LeakyReLU(),
                nn.Linear(self.in_features //2, self.in_features),
                nn.Sigmoid()
                )
        )
        self.decoder = nn.Sequential(*modules)
      
    def encode(self, x):
        """ Encode input to mean and logvar."""
        result = self.encoder(x)
        mu = self.fc_mu(result)
        logvar = self.fc_var(result)

        return [mu, logvar]
    
    def reparameterise(self, mu, logvar):
        """ Reparameterise to sample."""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        
        return eps * std + mu

    def decode(self, z):
        """ Decode latent sampling to output."""
        result = self.decoder(z)

        return result

    def forward(self, x):
        """ Forward pass of the network."""
        # Check input matches in_features
        if x.shape[-1] != self.in_features:
            raise Exception(f'Input features must be equal to {self.in_features}!')
        
        # Encode input to mean and logvar
        mu, logvar = self.encode(x)

        # Reparameterise
        z = self.reparameterise(mu, logvar)

        # Decode
        return [self.decode(z), mu, logvar]

# Loss function
class VAELoss(nn.Module):
    """ Loss function for VAE
        using BCE loss and KL 
        divergence.
    """
    def __init__(self):
        super(VAELoss, self).__init__()
        self.criterion = F.binary_cross_entropy

    def forward(self, outputs, inputs, mu, logvar, Beta=5):   
        recon_loss = self.criterion(outputs, inputs, reduction='sum')
        kl = 0.5 * torch.sum(-1 - logvar + mu.pow(2) + logvar.exp())

        return recon_loss, kl, recon_loss + kl

## **2. Train Standard VAE**

In [15]:
# Initiliase standard model
StandardVAE = VAE(in_features=N_FEATURES, latent_dim=5).to(device)
params = sum(p.numel() for p in StandardVAE.parameters() if p.requires_grad)
print("Total number of parameters for the Standard VAE: {}\n".format(params))
print("Standard VAE architecture:")
print(StandardVAE)

# Optimiser and loss
optimizer = torch.optim.Adam(StandardVAE.parameters(), lr=1e-3)
criterion = VAELoss()

Total number of parameters for the Standard VAE: 524

Standard VAE architecture:
VAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=19, out_features=9, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
    )
  )
  (fc_mu): Linear(in_features=9, out_features=5, bias=True)
  (fc_var): Linear(in_features=9, out_features=5, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=5, out_features=9, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=9, out_features=19, bias=True)
      (3): Sigmoid()
    )
  )
)


In [16]:
# Initialise loss dictionary
losses = {'Reconstruction': [],
          'KL': [],
          'Training': []
          }

# Begin training
print('Beginning Standard VAE training...\n')
StandardVAE.train()

# Loop over epochs
for epoch in range(N_EPOCHS):
    # Running loss containers
    running_recon_loss = 0.0
    running_kl_loss = 0.0
    running_train_loss = 0.0

    # Loop over batches
    with tqdm.tqdm(train_dataloader, unit="batch") as tepoch: 
        for batch_idx, (data, label) in enumerate(tepoch):
          # Batch tensor
          batch_tensor = data.to(device)

          # Compute reconstructions
          results, mu, logvar = StandardVAE(batch_tensor)

          # Loss
          recon_loss, kl_loss, train_loss = criterion(results, batch_tensor,
                                                      mu=mu, logvar=logvar)

          # Backpropagation based on the loss
          optimizer.zero_grad()
          train_loss.backward()
          optimizer.step()

          # Update loss
          running_recon_loss += recon_loss.item()
          running_kl_loss += kl_loss.item()
          running_train_loss += train_loss.item()

          # Log
          if batch_idx % 20 == 0:
            tepoch.set_description(f"Epoch {epoch+1}")
            tepoch.set_postfix(loss=train_loss.item())

        # Average epoch loss
        losses['Reconstruction'].append(running_recon_loss/batch_idx+1)
        losses['KL'].append(running_kl_loss/batch_idx+1)
        losses['Training'].append(running_train_loss/batch_idx+1)

Beginning Standard VAE training...



Epoch 25: 100%|██████████| 12990/12990 [00:11<00:00, 1110.72batch/s, loss=-1.66e+5]
